In [1]:
from rehearsal_scheduler.constraints import DayOfWeekConstraint, TimeOnDayConstraint

# constraint_text = "sun  1700-1900"
# constraint_text = "sun before 1700"
# constraint_text = "sun after 1700"
# constraint_text = "f before 9am"
constraint_text = "f before 9"
# constraint_text = "sat before 10am"
constraint_text = "m"

In [2]:
from rehearsal_scheduler.grammar import constraint_parser
constraint_text = "m"
parser = constraint_parser(debug=True)
result = parser.parse(constraint_text)
print(f"\nres = {result}")

day_spec <class 'str'>: 'monday'
temporal_constraint  <class 'str'>: 'monday' <class 'NoneType'>: None
constraint <class 'rehearsal_scheduler.constraints.DayOfWeekConstraint'>: DayOfWeekConstraint(day_of_week='monday')

res = (DayOfWeekConstraint(day_of_week='monday'),)


In [3]:
import inspect
from datetime import time

from lark import Lark, Transformer, v_args

DEBUG = False


def type_and_value(obj):  # pragma: no cover
    """Helper for debugging: returns the type and value of an object."""
    if DEBUG:  # pragma: no cover
        return f"{type(obj)}: {repr(obj)}"
    return ""  # pragma: no cover


GRAMMAR = r"""
    ?start: tod
    
    tod: std_time | military_time
    
    std_time: HOUR_STD (":" MINUTE)? AM_PM?
    military_time : (MORNING_MIL | AFTERNOON_MIL) MINUTE
    
    // TERMINALS
    
    HOUR_STD: "1" | "2" | "3" | "4" | "5" | "6" | "7" | "8" | "9" | "10" | "11" | "12"

    MORNING_MIL:   "00" | "01" | "02" | "03" | "04" | "05" | "06" | "07" | "08" | "09" | "10" | "11" 
    AFTERNOON_MIL: "12" | "13" | "14" | "15" | "16" | "17" | "18" | "19" | "20" | "21" | "22" | "23"

    MINUTE : "0" "0" | "0" "1" | "0" "2" | "0" "3" | "0" "4" | "0" "5" | "0" "6" | "0" "7" | "0" "8" | "0" "9" 
           | "1" "0" | "1" "1" | "1" "2" | "1" "3" | "1" "4" | "1" "5" | "1" "6" | "1" "7" | "1" "8" | "1" "9" 
           | "2" "0" | "2" "1" | "2" "2" | "2" "3" | "2" "4" | "2" "5" | "2" "6" | "2" "7" | "2" "8" | "2" "9" 
           | "3" "0" | "3" "1" | "3" "2" | "3" "3" | "3" "4" | "3" "5" | "3" "6" | "3" "7" | "3" "8" | "3" "9" 
           | "4" "0" | "4" "1" | "4" "2" | "4" "3" | "4" "4" | "4" "5" | "4" "6" | "4" "7" | "4" "8" | "4" "9" 
           | "5" "0" | "5" "1" | "5" "2" | "5" "3" | "5" "4" | "5" "5" | "5" "6" | "5" "7" | "5" "8" | "5" "9"

    AM_PM: "am"i | "pm"i
"""


@v_args(inline=True)
class ConstraintTransformer(Transformer):
    """Transforms the parsed Lark tree into constraint objects."""

    def INT(self, i):
        return int(i)

    def MORNING_MIL(self, h):
        return int(h)

    def AFTERNOON_MIL(self, h):
        return int(h)

    def HOUR_STD(self, h):
        return int(h)

    def MINUTE(self, m):
        return int(m)

    def AM_PM(self, am_pm):
        if DEBUG:  # pragma: no cover
            print(f"{inspect.stack()[0][3]} {type_and_value(am_pm)}")
        return am_pm.lower()

    @v_args(inline=False)
    def military_time(self, children):
        if DEBUG:  # pragma: no cover
            print(f"{inspect.stack()[0][3]} {type_and_value(children)}")
        hour, minute = children
        return (hour, minute)

    @v_args(inline=False)
    def tod(self, children):
        if DEBUG:  # pragma: no cover
            print(f"{inspect.stack()[0][3]} {type_and_value(children)}")
        h, m = children[0]
        return time(h, m)

    @v_args(inline=False)
    def std_time(self, children):
        """possible inputs
        h        : 10,11 = am, everything else is pm, m = 0
        h ampm   : h 0 ampm
        h m      : same rule for hours as h
        h m ampm :
        """
        if DEBUG:  # pragma: no cover
            print(f"{inspect.stack()[0][3]} {type_and_value(children)}")
        if len(children) == 3:
            h, m, fmt = children
            if fmt == "pm":
                h += 12
            return (h, m)
        if len(children) == 2:
            h, opt = children
            if isinstance(opt, str):
                if opt == "pm":
                    h += 12
                return (h, 0)
            else:
                if h not in [10, 11]:
                    h += 12
                return (h, opt)

        h = children[0]
        if h not in [10, 11]:
            h += 12
        return (h, 0)


def constraint_parser(grammar=GRAMMAR, debug=False):
    constraint_transformer = ConstraintTransformer()
    global DEBUG
    DEBUG = debug
    return Lark(grammar, parser="lalr", transformer=constraint_transformer, debug=debug)


parser = constraint_parser(debug=True)
result = parser.parse("1000")
print(f"\nres = {result}")
result = parser.parse("10am")
print(f"\nres = {result}")

military_time <class 'list'>: [10, 0]
tod <class 'list'>: [(10, 0)]

res = 10:00:00


UnexpectedToken: Unexpected token Token('AM_PM', 'am') at line 1, column 3.
Expected one of: 
	* MINUTE
Previous tokens: [Token('MORNING_MIL', '10')]


In [4]:
import lark
from rehearsal_scheduler.grammar_new import constraint_parser

constraint_text = "m 2401-2501"
parser = constraint_parser(debug=True)
exc = None
try:
    result = parser.parse(constraint_text)
    print(f"\nres = {result}")
except (
    lark.exceptions.UnexpectedToken,
    lark.exceptions.UnexpectedCharacters,
    SyntaxError,
) as e:
    exc = e
    print(f"Syntax error {e}")
except Exception as e:
    print(type(e))
    print(f"Error {e}")

ModuleNotFoundError: No module named 'rehearsal_scheduler.grammar_new'

In [ ]:
from rehearsal_scheduler.grammar_new import constraint_parser

parser = constraint_parser(debug=True)
result = parser.parse(constraint_text)
print(f"\nres = {result}")

In [ ]:
type(exc)

In [ ]:
exc.line, exc.expected, exc.token, exc.pos_in_stream

In [ ]:
print(f"found {exc.line} at position {exc.pos_in_stream + 1} in '{constraint_text}'")

In [ ]:
exc.line

In [ ]:
print(exc.UnexpectedToken())
help(exc)

In [ ]:
int("09")

In [ ]:
(
    ("a") == ("a",),
    len("a"),
    len(
        "a",
    ),
)

In [ ]:
def check_for_conflict(
    rehearsal: RehearsalSlot, dancer_constraints: list[UnavailabilityConstraint]
) -> list[TimeInterval]:
    """Checks for conflicts and returns a list of conflicting intervals."""
    all_conflicts = []
    for constraint in dancer_constraints:
        # print(f"constraint is {constraint}")
        conflicting_intervals = constraint.get_conflicting_intervals(rehearsal)
        all_conflicts.extend(conflicting_intervals)
    return all_conflicts

# Functions


In [ ]:
from rehearsal_scheduler.constraints import RehearsalSlot

In [ ]:
import dataclasses
from abc import ABC, abstractmethod
from typing import TypeAlias

TimeInterval: TypeAlias = tuple[int, int]


class UnavailabilityConstraint(ABC):
    """Abstract base class for all dancer unavailability constraints."""

    @abstractmethod
    def get_conflicting_intervals(self, slot: RehearsalSlot) -> list[TimeInterval]:
        pass


@dataclasses.dataclass(frozen=True)
class DayOfWeekConstraint(UnavailabilityConstraint):
    """Represents an unavailability for an entire day of the week."""

    day_of_week: str

    def get_conflicting_intervals(self, slot: RehearsalSlot) -> list[TimeInterval]:
        if self.day_of_week == slot.day_of_week:
            return [(slot.start_time, slot.end_time)]
        return []


@dataclasses.dataclass(frozen=True)
class TimeOnDayConstraint(UnavailabilityConstraint):
    """
    Represents an unavailability for a specific time interval on a given day of the week.
    e.g., "Mondays from 900 to 1200"
    """

    day_of_week: str  # e.g., 'monday'
    start_time: int  # Military time, e.g., 900
    end_time: int  # Military time, e.g., 1200

    def get_conflicting_intervals(self, slot: RehearsalSlot) -> list[TimeInterval]:
        """
        Checks for conflicts if the day matches AND the time intervals overlap.
        """
        if self.day_of_week != slot.day_of_week:
            return []  # No conflict if it's not on the right day.

        # Classic interval overlap check: max(starts) < min(ends)
        overlap_start = max(self.start_time, slot.start_time)
        overlap_end = min(self.end_time, slot.end_time)

        if overlap_start < overlap_end:
            # A conflict exists! Return the actual interval of the conflict.
            return [(overlap_start, overlap_end)]

        return []

In [ ]:
from typing import List, Optional, Tuple

from lark import Lark
from lark.exceptions import LarkError


# Assuming your SemanticValidationError is defined as before
class SemanticValidationError(ValueError):
    """Raised when the syntax is valid but the meaning is not."""

    pass


def validate_and_parse_all_constraints(
    parser: Lark, text: str
) -> Tuple[bool, Optional[List], Optional[str]]:
    """
    Parses a comma-separated string of constraints, validating each one individually
    and collecting ALL errors found.

    Returns:
        A tuple of (success, results, error_message).
        - If ALL constraints are valid, (True, [list_of_constraints], None).
        - If ANY constraint is invalid, (False, None, "multi-line error report").
    """
    parsed_constraints = []
    error_messages = []

    constraint_chunks = [chunk.strip() for chunk in text.split(",") if chunk.strip()]

    if not constraint_chunks:
        return (True, [], None)

    for chunk in constraint_chunks:
        try:
            parsed_result = parser.parse(chunk)

            # THE FIX: Use .append() to add the single object returned by the parser.
            # .extend() would fail with a TypeError because the parsed_result is not a list.
            parsed_constraints.append(parsed_result)

        except LarkError:
            msg = f"- Constraint '{chunk}' is not grammatically valid."
            error_messages.append(msg)

        except SemanticValidationError as e:
            msg = f"- Constraint '{chunk}' is not logically valid: {e}"
            error_messages.append(msg)

        except Exception as e:
            # This block will now only catch truly unexpected errors.
            msg = f"- An unexpected error occurred on constraint '{chunk}': {e}"
            error_messages.append(msg)

    if error_messages:
        header = "Found one or more invalid constraints:"
        # Corrected usage of '\n' for newlines
        final_error_message = f"{header}\n" + "\n".join(error_messages)
        return (False, None, final_error_message)
    else:
        return (True, parsed_constraints, None)

## Grammar Specification

In [ ]:
from typing import Optional, TypeAlias

from lark import Lark, Transformer, v_args


# --- A custom exception for semantic errors ---
class SemanticValidationError(ValueError):
    """Raised when the syntax is valid but the meaning is not."""

    pass

## executable code

## Conflicts

In [ ]:
rehearsal_1 = RehearsalSlot(
    date(2024, 9, 4), "wednesday", 1800, 1930
)  # Should conflict
rehearsal_2 = RehearsalSlot(
    date(2024, 9, 5), "thursday", 1900, 2100
)  # Should NOT conflict

conflicts_1 = check_for_conflict(rehearsal_1, parsed_constraints)
print(f"Checking Rehearsal 1 (Wednesday): Found conflicts: {conflicts_1}")

conflicts_2 = check_for_conflict(rehearsal_2, parsed_constraints)
print(f"Checking Rehearsal 2 (Thursday): Found conflicts: {conflicts_2}")

In [ ]:
# Assuming all the classes (RehearsalSlot, constraints) and the transformer are defined above...

# --- Main Execution Logic ---
try:
    # 1. Instantiate the parser with the new grammar and transformer
    constraint_parser = Lark(
        constraint_grammar,
        start="start",
        parser="lalr",
        transformer=ConstraintTransformer(),
    )

    # 2. Define a complex unavailability string
    dancer_unavailability_string = "mo until 12, w 2-4, fri after 5pm, sunday"

    # 3. Parse the string
    parsed_constraints = constraint_parser.parse(dancer_unavailability_string)

    print(f"Input string: '{dancer_unavailability_string}'")
    print("Parsed constraint objects:")
    for c in parsed_constraints:
        print(f"  - {c}")
    print("\n")

    # 4. Define rehearsals to check
    rehearsal_ok = RehearsalSlot(
        date(2024, 9, 2), "monday", 1300, 1500
    )  # OK, it's after 1200
    rehearsal_conflict_1 = RehearsalSlot(
        date(2024, 9, 2), "monday", 1100, 1230
    )  # CONFLICT, overlaps with "until 1200"
    rehearsal_conflict_2 = RehearsalSlot(
        date(2024, 9, 4), "wednesday", 1500, 1700
    )  # CONFLICT, overlaps with 1400-1600
    rehearsal_conflict_3 = RehearsalSlot(
        date(2024, 9, 8), "sunday", 1000, 1100
    )  # CONFLICT, unavailable all day Sunday

    # 5. Check for conflicts
    print(f"Checking {rehearsal_ok.day_of_week} at {rehearsal_ok.start_time}:")
    print(f"  -> Conflicts: {check_for_conflict(rehearsal_ok, parsed_constraints)}\n")

    print(
        f"Checking {rehearsal_conflict_1.day_of_week} at {rehearsal_conflict_1.start_time}:"
    )
    print(
        f"  -> Conflicts: {check_for_conflict(rehearsal_conflict_1, parsed_constraints)}\n"
    )

    print(
        f"Checking {rehearsal_conflict_2.day_of_week} at {rehearsal_conflict_2.start_time}:"
    )
    print(
        f"  -> Conflicts: {check_for_conflict(rehearsal_conflict_2, parsed_constraints)}\n"
    )

    print(
        f"Checking {rehearsal_conflict_3.day_of_week} at {rehearsal_conflict_3.start_time}:"
    )
    print(
        f"  -> Conflicts: {check_for_conflict(rehearsal_conflict_3, parsed_constraints)}\n"
    )

    # 6. Test the validation
    print("Testing invalid input 'th after 25'...")
    invalid_string = "th after 25"
    constraint_parser.parse(invalid_string)

except Exception as e:
    print(f"Successfully caught expected error: {e}")

In [ ]:
# Assuming 'constraint_parser' is your Lark instance with the ConstraintTransformer

# A string with a syntactic error ("zulu") and a semantic error ("th after 99")
test_text_multiple_errors = "Monday, zulu, th after 99, wed"

print(f"Checking: '{test_text_multiple_errors}'")
ok, constraints, errors = validate_and_parse_all_constraints(
    constraint_parser, test_text_multiple_errors
)

if ok:
    print("\nSuccess! Parsed constraints:")
    for c in constraints:
        print(f"  - {c}")
else:
    print(f"\nFailure! Reasons:\n{errors}")

print("-" * 20)

In [ ]:
# A fully valid string for comparison
test_text_valid = "mo until 12, w 2-4, fri after 5pm"
print(f"Checking: '{test_text_valid}'")
ok, constraints, errors = validate_and_parse_all_constraints(
    constraint_parser, test_text_valid
)

if ok:
    print("\nSuccess! Parsed constraints:")
    for c in constraints:
        print(f"  - {c}")
else:
    print(f"\nFailure! Reasons:\n{errors}")

In [ ]:
# A string with a syntactic error ("zulu") and a semantic error ("th after 99")
test_text_multiple_errors = "Monday, zulu, th after 99, th after 5, wed"

print(f"Checking: '{test_text_multiple_errors}'")
ok, constraints, errors = validate_and_parse_all_constraints(
    constraint_parser, test_text_multiple_errors
)

if ok:
    print("\nSuccess! Parsed constraints:")
    # Assuming your constraints have a __str__ or __repr__ method
    for c in constraints:
        print(f"  - {c}")
else:
    print(f"\nFailure! Reasons:\n{errors}")

In [11]:
# This grammar validates ranges and solves ambiguity!
GRAMMAR_V3 = r"""
    ?start: tod
    
    tod: std_time | military_time

    // RULES
    std_time: STD_HOUR (":" MINUTE)? AM_PM?
    military_time: MILITARY_TIME

    // TERMINALS -- These now perform the validation!
    
    // Matches a full, valid 24-hour time string like "2359" or "0830"
    MILITARY_TIME: /([01]\d|2[0-3])[0-5]\d/ 
    
    // Matches a valid 12-hour format hour, like "12" or "9"
    STD_HOUR: /1[0-2]|[1-9]/

    // Matches a valid minute, like "05" or "59"
    MINUTE: /[0-5]\d/

    AM_PM: "am"i | "pm"i
    
    %ignore /\s+/
"""

In [14]:
from datetime import time

from lark import Lark, Transformer, v_args


@v_args(inline=True)
class ConstraintTransformerV2(Transformer):
    """Transforms the parsed Lark tree into constraint objects."""

    def STD_HOUR(self, h):
        return int(h)

    def MINUTE(self, m):
        return int(m)

    def AM_PM(self, am_pm):
        return am_pm.lower()

    # This method receives the validated 4-digit string from MILITARY_TIME
    def military_time(self, token):
        hour = int(token[0:2])
        minute = int(token[2:4])
        return (hour, minute)

    @v_args(inline=False)
    def tod(self, children):
        if not children:
            return None
        h, m = children[0]
        return time(h, m)

    # This logic is mostly the same as before, just cleaner.
    @v_args(inline=False)
    def std_time(self, children):
        h = children[0]
        m = 0
        fmt = None

        # Unpack the optional minute and am/pm
        if len(children) > 1:
            if isinstance(children[1], str):  # It's AM/PM
                fmt = children[1]
            else:  # It's a minute
                m = children[1]

        if len(children) > 2:  # Must be AM/PM if minute was also present
            fmt = children[2]

        # Apply AM/PM logic
        if fmt == "pm" and h != 12:
            h += 12
        elif fmt == "am" and h == 12:  # Handle midnight case: 12am is 00:00
            h = 0
        elif fmt is None:  # No am/pm specified
            # Your original logic for inferring am/pm
            if h in [1, 2, 3, 4, 5, 6, 7, 12]:
                h += 12
            if h == 24:  # 12pm becomes 24, should be 12
                h = 12
        return (h, m)

In [15]:
from lark import Lark, UnexpectedToken

# ... (define the transformer and other helpers here) ...


def constraint_parser(grammar=GRAMMAR_V3, debug=False):
    # Use the new Transformer
    constraint_transformer = ConstraintTransformerV2()
    global DEBUG
    DEBUG = debug
    return Lark(grammar, parser="lalr", transformer=constraint_transformer, debug=debug)


parser = constraint_parser()

# --- This will now fail, as desired! ---
try:
    parser.parse("2500")
except UnexpectedToken as e:
    print("Correctly failed to parse '2500'!")
    print(e)

# --- This will also fail ---
try:
    parser.parse("1061")
except UnexpectedToken as e:
    print("\nCorrectly failed to parse '1061'!")
    print(e)

# --- These will succeed ---
print("\n--- Parsing '1000' ---")
result = parser.parse("1000")
print(f"res = {result}")

print("\n--- Parsing '10am' ---")
result = parser.parse("10am")
print(f"res = {result}")

Correctly failed to parse '2500'!
Unexpected token Token('STD_HOUR', '5') at line 1, column 2.
Expected one of: 
	* $END
	* AM_PM
	* COLON
Previous tokens: [Token('STD_HOUR', '2')]


Correctly failed to parse '1061'!
Unexpected token Token('STD_HOUR', '6') at line 1, column 3.
Expected one of: 
	* $END
	* AM_PM
	* COLON
Previous tokens: [Token('STD_HOUR', '10')]


--- Parsing '1000' ---
res = 10:00:00

--- Parsing '10am' ---
res = 10:00:00


# Notes on EBNF

## The Jupyternaut Prescription for Lark Transformers

Here is a concise guide with minimal examples to serve as your mental model. Think of this as the "Standard Model" for connecting your grammar to your transformer.

The fundamental principle is: **For every possible path a rule can take, there must be a corresponding method in the transformer to handle it.**

### Case 1: The Default Rule (No Alias)

This is the simplest case. The method name in your transformer **must exactly match** the rule name in your grammar.

*   **Grammar Snippet (EBNF):**
    ```lark
    time_range: time "-" time
    ```
*   **How to Identify:** A simple rule name (`time_range`) defined with a colon (`:`).
*   **Transformer Snippet (Python):**
    ```python
    class MyTransformer(Transformer):
        # Method name matches the rule name
        def time_range(self, children):
            # children will be a list of the processed parts, e.g., [900, 1700]
            start_time, end_time = children
            return (start_time, end_time)
    ```

### Case 2: The Alias Rule (The `->` Arrow)

The alias (`->`) completely overrides the default behavior. Lark will ignore the rule name and look for a method matching the alias name.

*   **Grammar Snippet (EBNF):**
    ```lark
    time_range: time "-" time -> build_explicit_range
    ```
*   **How to Identify:** Look for the arrow `->` followed by a new name (`build_explicit_range`).
*   **Transformer Snippet (Python):**
    ```python
    class MyTransformer(Transformer):
        # Method name matches the ALIAS name
        def build_explicit_range(self, children):
            start_time, end_time = children
            return (start_time, end_time)
    ```

### Case 3: The "Or" Rule (Multiple Branches)

This is the one that caught you, and it's just a combination of the first two cases. You must treat each branch (separated by a `|`) as its own independent path.

*   **Grammar Snippet (EBNF):**
    ```lark
    range: time "-" time -> build_explicit_range  // Path 1 (Alias)
         | "until"i time                        // Path 2 (Default)
    ```
*   **How to Identify:** Look for the vertical bar `|` separating different patterns. Check each pattern for an alias.
*   **Transformer Snippet (Python):** You need one method for each path.
    ```python
    class MyTransformer(Transformer):
        # Method for Path 1, matching the alias
        def build_explicit_range(self, children):
            start_time, end_time = children
            return (start_time, end_time)

        # Method for Path 2, matching the RULE name ("range")
        def range(self, children):
            # This branch only has one child: the processed `time`
            end_time = children[0]
            return (0, end_time) # from start of day until the time
    ```

### Case 4: Transforming Terminals (The Uppercase Tokens)

You can also write methods for the "atoms" of your grammar—the Terminals (usually in `ALL_CAPS`). This is extremely useful for converting raw text into useful data types like numbers or dates.

*   **Grammar Snippet (EBNF):**
    ```lark
    INT: /[0-9]+/
    ```
*   **How to Identify:** A token name in all caps.
*   **Transformer Snippet (Python):**
    ```python
    class MyTransformer(Transformer):
        # Method name matches the Terminal name
        def INT(self, number_token):
            # The argument is the token object itself
            # We convert its string value to a Python integer
            return int(number_token.value)
    ```